# Détection en Temps Réel

Ce notebook teste la détection de panneaux en temps réel avec une webcam.

## ⚠️ IMPORTANT - Ordre d'Exécution

**Vous DEVEZ exécuter les cellules dans l'ordre suivant :**

1. **Cellule 1** : Imports (exécutez d'abord)
2. **Cellule 3** : "1. Charger le Modèle" (OBLIGATOIRE avant les autres méthodes)
3. **Ensuite** : Vous pouvez utiliser n'importe quelle méthode (1, 2, 3 ou 4)

## Objectifs
- Charger un modèle entraîné
- Détecter les panneaux dans une image
- Tester avec la webcam (optionnel)


In [17]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent / "src"))

import numpy as np
import matplotlib.pyplot as plt
import cv2
from src.data_loader import GTSRBDataLoader
from src.detector import RealTimeDetector
from src.model import TrafficSignClassifier

# Configuration - Chemins vers les données et le modèle
DATA_PATH = "../data"
MODEL_PATH = "../models/traffic_sign_cnn.h5"

print("✅ Imports réussis")
print(f"📁 DATA_PATH: {DATA_PATH}")
print(f"📁 MODEL_PATH: {MODEL_PATH}")


✅ Imports réussis
📁 DATA_PATH: ../data
📁 MODEL_PATH: ../models/traffic_sign_cnn.h5


## 1. Charger le Modèle

⚠️ **OBLIGATOIRE** : Exécutez cette cellule en premier ! Elle initialise le détecteur nécessaire pour toutes les autres méthodes.


In [18]:
# Vérifier si le modèle existe et initialiser le détecteur
from pathlib import Path

# Fonction pour trouver le modèle automatiquement
def find_model():
    """Trouve le modèle disponible"""
    # Chemins possibles pour le modèle
    possible_paths = [
        Path(MODEL_PATH),  # Chemin défini dans la cellule 1
        Path("../models/traffic_sign_cnn.h5"),
        Path("../models/traffic_sign_resnet.h5"),
        Path("models/traffic_sign_cnn.h5"),
        Path("models/traffic_sign_resnet.h5"),
    ]
    
    for model_path in possible_paths:
        if model_path.exists():
            return str(model_path)
    return None

# Initialiser la variable globale detector à None
if 'detector' not in globals():
    detector = None

# Trouver le modèle
model_path = find_model()

if model_path is None:
    print(f"❌ ERREUR: Aucun modèle trouvé!")
    print("\nModèles recherchés:")
    print("  - models/traffic_sign_cnn.h5")
    print("  - models/traffic_sign_resnet.h5")
    print("\nPour résoudre ce problème:")
    print("1. Entraînez d'abord un modèle avec le notebook 03_model_training.ipynb")
    print("2. Ou placez un modèle dans le dossier models/")
    print("\n⚠️  IMPORTANT: Vous devez exécuter cette cellule avec succès avant d'utiliser les autres méthodes!")
else:
    try:
        print(f"✅ Modèle trouvé: {model_path}")
        
        # Vérifier DATA_PATH
        data_path_obj = Path(DATA_PATH)
        if not data_path_obj.exists():
            # Essayer d'autres chemins
            alt_paths = [Path("../data"), Path("data")]
            for alt_path in alt_paths:
                if alt_path.exists():
                    DATA_PATH = str(alt_path)
                    break
        
        # Charger les noms de classes
        loader = GTSRBDataLoader(DATA_PATH)
        class_names = loader.get_class_names()
        
        # Créer le détecteur
        detector = RealTimeDetector(model_path, class_names)
        print("✅ Détecteur initialisé avec succès!")
        print(f"✅ {len(class_names)} classes de panneaux disponibles")
    except Exception as e:
        print(f"❌ ERREUR lors de l'initialisation: {e}")
        print("Vérifiez que toutes les dépendances sont installées")
        import traceback
        traceback.print_exc()
        detector = None


✅ Modèle trouvé: models\traffic_sign_cnn.h5


✅ Détecteur initialisé avec succès!
✅ 21 classes de panneaux disponibles


## 2. Test sur une Image - Chargement Interactif

Vous pouvez charger une image de plusieurs façons :
1. **Sélectionner un fichier** avec le widget
2. **Entrer un chemin manuellement**
3. **Télécharger depuis une URL**


In [19]:
# MÉTHODE 1 : Sélectionner un fichier avec un widget (Recommandé)
from ipywidgets import FileUpload, Button, Output, VBox
from IPython.display import display, clear_output
import io

upload = FileUpload(accept='.jpg,.jpeg,.png,.ppm,.bmp', multiple=False)
output = Output()

def on_upload_change(change):
    # Vérifier que le détecteur est initialisé
    if 'detector' not in globals() or detector is None:
        with output:
            clear_output()
            print("❌ ERREUR: Le détecteur n'est pas initialisé!")
            print("\nVeuillez d'abord exécuter la cellule '1. Charger le Modèle' avec succès.")
        return
    
    if upload.value:
        with output:
            clear_output()
            # Récupérer le fichier uploadé
            uploaded_file = list(upload.value.values())[0]
            image_data = uploaded_file['content']
            
            # Convertir en image numpy
            nparr = np.frombuffer(image_data, np.uint8)
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            
            if img is not None:
                # Sauvegarder temporairement
                temp_path = "temp_uploaded_image.jpg"
                cv2.imwrite(temp_path, img)
                
                # Détecter
                result = detector.detect_from_image(temp_path)
                
                # Afficher
                plt.figure(figsize=(14, 8))
                plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
                plt.title("Détection sur Image Uploadée", fontsize=14)
                plt.axis('off')
                plt.tight_layout()
                plt.show()
                
                # Nettoyer
                import os
                if os.path.exists(temp_path):
                    os.remove(temp_path)
            else:
                print("Erreur: Impossible de charger l'image")

upload.observe(on_upload_change, names='value')

print("📁 Méthode 1 : Sélectionnez un fichier image ci-dessous")
display(upload, output)


📁 Méthode 1 : Sélectionnez un fichier image ci-dessous


FileUpload(value=(), accept='.jpg,.jpeg,.png,.ppm,.bmp', description='Upload')

Output()

### Méthode 2 : Entrer un chemin manuellement


In [ ]:
# MÉTHODE 2 : Entrer un chemin manuellement
from ipywidgets import Text, Button, Output, HBox
from IPython.display import display, clear_output

# Créer les widgets
path_input = Text(
    value='../data/Test/00000.ppm',
    placeholder='https://e7.pngegg.com/pngimages/31/875/png-clipart-germany-traffic-sign-overtaking-almanya-daki-otoyollar-driving-driving-truck-logo.png',
    description='Chemin:',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

output2 = Output()

def on_button_click(b):
    # Vérifier que le détecteur est initialisé
    if 'detector' not in globals() or detector is None:
        with output2:
            clear_output()
            print("❌ ERREUR: Le détecteur n'est pas initialisé!")
            print("\nVeuillez d'abord exécuter la cellule '1. Charger le Modèle' avec succès.")
        return
    
    image_path = path_input.value
    with output2:
        clear_output()
        from pathlib import Path
        
        if Path(image_path).exists():
            print(f"Chargement de: {image_path}")
            result = detector.detect_from_image(image_path)
            
            # Afficher le résultat
            plt.figure(figsize=(14, 8))
            plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
            plt.title(f"Détection sur: {Path(image_path).name}", fontsize=14)
            plt.axis('off')
            plt.tight_layout()
            plt.show()
        else:
            print(f"❌ Fichier non trouvé: {image_path}")
            print("Vérifiez que le chemin est correct")

button = Button(description='Charger et Détecter', button_style='primary')
button.on_click(on_button_click)

print("📝 Méthode 2 : Entrez un chemin vers une image")
display(HBox([path_input, button]), output2)


📝 Méthode 2 : Entrez un chemin vers une image


Output()

### Méthode 3 : Télécharger depuis une URL


In [ ]:
# MÉTHODE 3 : Télécharger depuis une URL
import urllib.request
from ipywidgets import Text, Button, Output
from IPython.display import display, clear_output

url_input = Text(
    value='',
    placeholder='https://example.com/image.jpg',
    description='URL:',
    style={'description_width': 'initial'},
    layout={'width': '600px'}
)

output3 = Output()

def on_url_button_click(b):
    # Vérifier que le détecteur est initialisé
    if 'detector' not in globals() or detector is None:
        with output3:
            clear_output()
            print("❌ ERREUR: Le détecteur n'est pas initialisé!")
            print("\nVeuillez d'abord exécuter la cellule '1. Charger le Modèle' avec succès.")
        return
    
    url = url_input.value
    if not url:
        with output3:
            clear_output()
            print("❌ Veuillez entrer une URL")
        return
    
    with output3:
        clear_output()
        try:
            print(f"Téléchargement depuis: {url}")
            
            # Télécharger l'image
            temp_path = "temp_url_image.jpg"
            urllib.request.urlretrieve(url, temp_path)
            
            # Charger et détecter
            result = detector.detect_from_image(temp_path)
            
            # Afficher
            plt.figure(figsize=(14, 8))
            plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
            plt.title("Détection sur Image depuis URL", fontsize=14)
            plt.axis('off')
            plt.tight_layout()
            plt.show()
            
            # Nettoyer
            import os
            if os.path.exists(temp_path):
                os.remove(temp_path)
                
        except Exception as e:
            print(f"❌ Erreur: {e}")
            print("Vérifiez que l'URL est valide et pointe vers une image")

url_button = Button(description='Télécharger et Détecter', button_style='success')
url_button.on_click(on_url_button_click)

print("🌐 Méthode 3 : Entrez l'URL d'une image")
display(HBox([url_input, url_button]), output3)


🌐 Méthode 3 : Entrez l'URL d'une image


Output()

### Méthode 4 : Test Simple avec Image du Dataset


In [ ]:
# MÉTHODE 4 : Test simple avec une image du dataset
# ⚠️ IMPORTANT: Exécutez d'abord la cellule "1. Charger le Modèle" !

if 'detector' not in globals() or detector is None:
    print("❌ ERREUR: Le détecteur n'est pas initialisé!")
    print("\nVeuillez d'abord exécuter la cellule '1. Charger le Modèle' avec succès.")
else:
    test_image_path = "../data/Test/00000.ppm"  # Modifiez selon votre dataset
    
    from pathlib import Path
    if Path(test_image_path).exists():
        print(f"Chargement de: {test_image_path}")
        result = detector.detect_from_image(test_image_path)
        
        # Afficher le résultat
        plt.figure(figsize=(14, 8))
        plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        plt.title("Détection sur Image du Dataset", fontsize=14)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    else:
        print(f"Image de test non trouvée: {test_image_path}")
        print("Utilisez une image de votre dataset de test")


## 3. Détection en Temps Réel avec Webcam

⚠️ **Note**: Exécutez cette cellule pour lancer la détection en temps réel. Appuyez sur 'q' pour quitter.


In [ ]:
# Lancer la détection en temps réel
# Décommentez la ligne suivante pour activer la webcam
# detector.run_detection(camera_index=0)


🔍 Testing: models/traffic_sign_cnn.h5
   Size: 29.8 MB
   Loading model...


✅ Model loaded successfully!
   Model type: Sequential
   Input shape: (None, 64, 64, 3)
   Output shape: (None, 43)
   Number of layers: 15
   Prediction shape: (1, 43)
   Can make predictions: ✅


True